## Recommendation System Spark, Trabajo Final

### Seminario de bases de datos 2020-1 UdeA

#### Estudiante: Carlos Alberto Benavidez Vega
#### Estudiante: Juan Camilo Rojas Ricaute

In [1]:
import pandas as pd

In [2]:
import findspark

In [3]:
findspark.init('C:\BigData\Spark')

In [4]:
findspark.init()
findspark.find()

'C:\\BigData\\Spark'

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import Row
import re 

In [6]:
conf = SparkConf().setAppName("SparkRecommendation").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

### Working With Rdds (ratings_rdd)

In [9]:
ratings_rdd=sc.textFile('ml-latest-small/ratings.csv') #File

In [10]:
ratings_rdd.take(10)

['userId,movieId,rating,timestamp',
 '1,1,4.0,964982703',
 '1,3,4.0,964981247',
 '1,6,4.0,964982224',
 '1,47,5.0,964983815',
 '1,50,5.0,964982931',
 '1,70,3.0,964982400',
 '1,101,5.0,964980868',
 '1,110,4.0,964982176',
 '1,151,5.0,964984041']

### Working with Rdds (movies_rdd)

In [11]:
movies_rdd=sc.textFile('ml-latest-small/movies.csv') #File

In [13]:
data_header_ratings = ratings_rdd.take(1)[0]
data_header_ratings

'userId,movieId,rating,timestamp'

In [14]:
ratings_data = ratings_rdd.filter(lambda line: line!=data_header_ratings)\
.map(lambda line: line.split(","))\
.map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
ratings_data.take(5)

[('1', '1', '4.0'),
 ('1', '3', '4.0'),
 ('1', '6', '4.0'),
 ('1', '47', '5.0'),
 ('1', '50', '5.0')]

In [15]:
data_header_movies=movies_rdd.take(1)[0]
data_header_movies

'movieId,title,genres'

In [16]:
movies_data=movies_rdd.filter(lambda line: line!=data_header_movies)\
.map(lambda line: line.split(","))\
.map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
movies_data.take(5)

[('1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 ('2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 ('3', 'Grumpier Old Men (1995)', 'Comedy|Romance'),
 ('4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'),
 ('5', 'Father of the Bride Part II (1995)', 'Comedy')]

### Splitting the rating Data

In [17]:
training, validation, test = ratings_data.randomSplit([6, 2, 2]) 

In [18]:
training.count()

60307

In [19]:
validation.count()

20315

In [20]:
test.count()

20214

In [21]:
validation.take(5)

[('1', '1', '4.0'),
 ('1', '6', '4.0'),
 ('1', '231', '5.0'),
 ('1', '235', '4.0'),
 ('1', '592', '4.0')]

In [22]:
test.take(5)

[('1', '70', '3.0'),
 ('1', '163', '5.0'),
 ('1', '333', '5.0'),
 ('1', '441', '4.0'),
 ('1', '553', '5.0')]

In [23]:
validation_for_predict_RDD = validation.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test.map(lambda x: (x[0], x[1]))

In [24]:
validation_for_predict_RDD.take(5)

[('1', '1'), ('1', '6'), ('1', '231'), ('1', '235'), ('1', '592')]

In [25]:
test_for_predict_RDD.take(5)

[('1', '70'), ('1', '163'), ('1', '333'), ('1', '441'), ('1', '553')]

In [26]:
from pyspark.mllib.recommendation import ALS

In [27]:
import math

In [28]:
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    #model = ALS.train(training, rank, seed=seed, iterations=iterations,lambda_=regularization_parameter)
    model = ALS.train(training, rank, iterations)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    #print 'For rank %s the RMSE is %s' % (rank, error)
    print ("RMSE is "+ str(error))
    
    if error < min_error:
        min_error = error
        best_rank = rank

print ("The best model was trained with rank "+ str(best_rank))


RMSE is 1.0809449650824248
RMSE is 1.1599092778722024
RMSE is 1.2493393873252636
The best model was trained with rank 4


In [29]:
rank = 4
numIterations = 10
model = ALS.train(training, rank, numIterations)
predictions = model.predictAll(validation_for_predict_RDD).map(lambda item: ((item[0], item[1]), item[2]))
rates_and_preds = validation.map(lambda row: ((int(row[0]), int(row[1])), float(row[2]))).join(predictions)
MSE = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))
   
 

Mean Squared Error = 1.1590555755168706


In [1]:
print("fin de la presentacion")

fin de la presentacion
